In [1]:
import numpy as np
import pandas as pd

In [2]:
# 2015 1,2,3,4 & 2016 1,2,3,4 & 2017 1,2,3,4 & 2018 1,2,3,4 & 2019 1,2,3,4 & 2020 1,2 = 총 22분기
apt_dir = r'\2020-2-OSSP-3P-1-main\2020-2-OSSP-3P-1-main\상주인구,아파트\상권_아파트.csv'
human_pop_dir = r'\2020-2-OSSP-3P-1-main\상주인구,아파트\상권_상주인구.csv'

apt = pd.read_csv( apt_dir )
human_pop = pd.read_csv( human_pop_dir )

In [3]:
# mean, median, knn 성능비교

apt = apt.drop(['Unnamed: 0','상권_구분_코드','상권_구분_코드_명','상권_코드_명'], axis=1)

In [4]:
apt

,기준_년_코드,기준_분기_코드,상권_코드,아파트_단지_수,아파트_평균_면적,아파트_평균_시가
0,2020,2,1001495,22,89,527347107
1,2020,1,1001495,22,89,527347107
2,2020,2,1001494,5,39,112097903
3,2020,1,1001494,5,39,112097903
4,2020,2,1001493,8,53,237967427
...,...,...,...,...,...,...
29309,2015,1,1000002,5,18,84110417
29310,2015,4,1000001,27,69,186139469
29311,2015,3,1000001,27,69,186139469
29312,2015,2,1000001,27,69,186139469


In [6]:
# one hot embedding
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder()
x = onehot.fit_transform(apt['상권_코드'].values.reshape(-1,1)).toarray()

onehot_df = pd.DataFrame(x, columns = ["상권코드_"+str(int(i)) for i in range(x.shape[1])])
apt_o_hot = pd.concat([apt, onehot_df], axis=1)

In [7]:
apt_o_hot = apt_o_hot.drop(['상권_코드'], axis=1)
apt_o_hot

,기준_년_코드,기준_분기_코드,아파트_단지_수,아파트_평균_면적,아파트_평균_시가,상권코드_0,상권코드_1,상권코드_2,상권코드_3,상권코드_4,...,상권코드_1348,상권코드_1349,상권코드_1350,상권코드_1351,상권코드_1352,상권코드_1353,상권코드_1354,상권코드_1355,상권코드_1356,상권코드_1357
0,2020,2,22,89,527347107,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2020,1,22,89,527347107,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2020,2,5,39,112097903,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2020,1,5,39,112097903,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2020,2,8,53,237967427,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29309,2015,1,5,18,84110417,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29310,2015,4,27,69,186139469,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29311,2015,3,27,69,186139469,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29312,2015,2,27,69,186139469,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 열 순서 재정렬

apt_o_hot = pd.concat([apt_o_hot.drop(['아파트_단지_수','아파트_평균_면적','아파트_평균_시가'],axis=1),apt_o_hot[['아파트_단지_수','아파트_평균_면적','아파트_평균_시가']]],axis=1)

In [10]:
from sklearn.model_selection import train_test_split

# 평균값으로 결측치 채우기, 중앙값으로 결측치 채우기, KNN으로 결측치 채우기 가운데 어떤 방법이 가장 RMSE가 낮은지 비교한다.
# 1. KNN

X_train, X_test, y_train, y_test = train_test_split(apt_o_hot.loc[:,'기준_년_코드':'상권코드_1357'], apt_o_hot[['아파트_단지_수','아파트_평균_면적','아파트_평균_시가']], test_size=0.3, random_state=42)

In [11]:
X_train

,기준_년_코드,기준_분기_코드,상권코드_0,상권코드_1,상권코드_2,상권코드_3,상권코드_4,상권코드_5,상권코드_6,상권코드_7,...,상권코드_1348,상권코드_1349,상권코드_1350,상권코드_1351,상권코드_1352,상권코드_1353,상권코드_1354,상권코드_1355,상권코드_1356,상권코드_1357
21693,2016,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6545,2019,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13166,2018,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26505,2015,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14945,2017,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,2016,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5390,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,2020,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15795,2017,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X_test

,기준_년_코드,기준_분기_코드,상권코드_0,상권코드_1,상권코드_2,상권코드_3,상권코드_4,상권코드_5,상권코드_6,상권코드_7,...,상권코드_1348,상권코드_1349,상권코드_1350,상권코드_1351,상권코드_1352,상권코드_1353,상권코드_1354,상권코드_1355,상권코드_1356,상권코드_1357
2982,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250,2020,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13078,2018,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3162,2019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2153,2020,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5333,2019,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12131,2018,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15035,2017,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20837,2016,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
y_train

,아파트_단지_수,아파트_평균_면적,아파트_평균_시가
21693,28,55,159980690
6545,28,66,142330073
13166,14,73,198986143
26505,59,54,112733495
14945,63,51,188459634
...,...,...,...
21575,25,56,106249649
5390,5,35,101205275
860,72,72,495041463
15795,22,54,162390420


In [14]:
y_test

,아파트_단지_수,아파트_평균_면적,아파트_평균_시가
2982,4,57,194462500
250,4,62,201350000
13078,9,63,350996120
3162,5,38,108034812
2153,41,55,128200271
...,...,...,...
5333,42,52,143664439
12131,14,40,86580189
15035,27,67,335208956
20837,21,49,108560671


In [15]:
import time
start = time.time()  # 시작 시간 저장
 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=150)
neigh.fit(X_train, y_train)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 6.728105306625366


In [16]:
start = time.time()  # 시작 시간 저장

y_pred = neigh.predict(X_test)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 76.48539710044861


In [17]:
y_pred = pd.DataFrame(y_pred)
y_pred

,0,1,2
0,3,59,194462500
1,4,55,22819512
2,1,59,38157895
3,3,59,67656206
4,3,50,61833333
...,...,...,...
8790,2,56,24512195
8791,1,49,86580189
8792,3,51,335208956
8793,1,55,16666667


In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
rmse_0 = mean_squared_error(y_test['아파트_단지_수'], y_pred[0])**0.5
rmse_1 = mean_squared_error(y_test['아파트_평균_면적'], y_pred[1])**0.5
rmse_2 = mean_squared_error(y_test['아파트_평균_시가'], y_pred[2])**0.5

In [20]:
print(rmse_0)
print(rmse_1)
print(rmse_2)

44.00101812869299
18.589925566696788
121969340.16138466


In [27]:
# 평균값 rmse 측정

y_mean = y_train.mean(axis=0)
y_mean_ = pd.DataFrame(index=range(0,8795), columns=['0', '1','2'])

In [28]:
rmse_0 = np.sqrt(((y_test['아파트_단지_수'] -y_mean[0]) ** 2).mean())
rmse_1 = np.sqrt(((y_test['아파트_단지_수'] -y_mean[1]) ** 2).mean())
rmse_2 = np.sqrt(((y_test['아파트_단지_수'] -y_mean[2]) ** 2).mean())

In [29]:
print(rmse_0)
print(rmse_1)
print(rmse_2)

34.24878484558119
43.757201149491785
174185708.65749305


In [30]:
# 중앙값 rmse 측정

y_med_0 = np.median(y_train['아파트_단지_수'])
y_med_1 = np.median(y_train['아파트_평균_면적'])
y_med_2 = np.median(y_train['아파트_평균_시가'])

In [31]:
rmse_0 = np.sqrt(((y_test['아파트_단지_수'] -y_med_0) ** 2).mean())
rmse_1 = np.sqrt(((y_test['아파트_단지_수'] -y_med_1) ** 2).mean())
rmse_2 = np.sqrt(((y_test['아파트_단지_수'] -y_med_2) ** 2).mean())

In [32]:
print(rmse_0)
print(rmse_1)
print(rmse_2)

35.608524635538146
41.95275233026003
142911451.2362772
